# Introduction to Transformers

The first part of this notebook is based on chapter 6 **Summarization** of the book **Natural Language Processing with Tranformers** and can be found [here](https://nbviewer.org/github/nlp-with-transformers/notebooks/blob/main/06_summarization.ipynb).

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, tqdm, time, torch
warnings.filterwarnings(action='ignore')
sys.path.insert(0, '../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import pipeline, set_seed
from transformers import (AutoModelForSeq2SeqLM, AutoTokenizer,
                          AutoModelForQuestionAnswering)
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer
from transformers.data.processors.squad import SquadV1Processor
from datasets import load_dataset, load_metric
import nltk
from nltk.tokenize import sent_tokenize
import string, re

set_seed(42)

## Functions

In [ ]:
def evaluate_summaries_baseline(dataset, metric, column_text='article', column_summary='abstract'):
  summaries = [three_sentence_summary(text) for text in dataset[column_text]]
  metric.add_batch(predictions=summaries, references=dataset[column_summary])    
  score = metric.compute()
  return score

def chunks(list_of_elements, batch_size):
  """
  Yield successive batch-sized chunks from list_of_elements.
  """
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=8, device=device, column_text='article',
                               column_summary='abstract'):
  article_batches = list(chunks(dataset[column_text], batch_size))
  target_batches = list(chunks(dataset[column_summary], batch_size))
  for article_batch, target_batch in tqdm_notebook(zip(article_batches, target_batches), total=len(article_batches)):
    inputs = tokenizer(article_batch, max_length=1024,  truncation=True, padding='max_length', return_tensors='pt')
    summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                               attention_mask=inputs['attention_mask'].to(device),
                               length_penalty=0.8, num_beams=8, max_length=128)

    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
                         for s in summaries]
    decoded_summaries = [d.replace('<n>', ' ') for d in decoded_summaries]
    metric.add_batch(predictions=decoded_summaries, references=target_batch)

  score = metric.compute()
  return score

def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

  return {'input_ids': input_encodings['input_ids'],
          'attention_mask': input_encodings['attention_mask'],
          'labels': target_encodings['input_ids']}

In [ ]:
def display_example(qid):    
  idx = qid_to_example_index[qid]
  q = examples[idx].question_text
  c = examples[idx].context_text
  a = [answer['text'] for answer in examples[idx].answers]

  print(f'Example {idx} of {len(examples)}\n---------------------')
  print(f"Q: {q}\n")
  print("Context:")
  print(c)
  print(f"\nTrue Answers:\n{a}")
  
def get_prediction(qid):
  # given a question id (qas_id or qid), load the example, get the model outputs and generate an answer
  question = examples[qid_to_example_index[qid]].question_text
  context = examples[qid_to_example_index[qid]].context_text

  inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1 

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer
  
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
  return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]
    
    return gold_answers

## HuggingFace's Summarization Pipeline

### Load Data

The dataset we are using for this task is the Pubmed Summarization dataset which consists of 119,924 pairs of articles and their corresponding abstracts.

This dataset can be found in the Hugging Face hub [here](https://huggingface.co/datasets/ccdv/pubmed-summarization).

In [ ]:
art_idx = 2
dataset = load_dataset('ccdv/pubmed-summarization')
print(f"Features: {dataset['train'].column_names}")

sample_text = dataset['train'][art_idx]
print(f"Article (excerpt of 500 characters, total length: {len(sample_text['article'])}):")
print(sample_text['article'][:500])
print(f"\nSummary (length: {len(sample_text['abstract'])}):")
print(sample_text['abstract'])

We limit the articles' length to 2000 characters to have the same input to all the models and due to memory restrictions.

In [ ]:
sample_text = dataset['train'][art_idx]['article'][:2000]
summaries = {}

### Generate Summaries using Different Models Models

#### Primitive Summarization - Just take first 3 sentences

In [ ]:
def three_sentence_summary(text):
  return '\n'.join(sent_tokenize(text)[:3])

summaries['baseline'] = three_sentence_summary(sample_text)

#### Summarization with GPT-2

By adding `TL;DR:` at the end of the article prompts the GPT-2 model to generate a summary instead to generating free text

In [ ]:
pipe = pipeline('text-generation', model='gpt2-xl')
gpt2_query = sample_text + '\nTL;DR:\n' 
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries['gpt2'] = '\n'.join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query) :]))

### T5

T5 transformer is a universal trasnformer architecture by formulating all tasks as text-to-text tasks. T5 checkpoints are trained ona mixture of unsupervised data (to resconstruct masked words) and supervised data for several tasks including summarization.

In [ ]:
pipe = pipeline('summarization', model='t5-large')
pipe_out = pipe(sample_text)
summaries['t5'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

### BART

BART also uses an encoder-decoder architecture and is trained to reconstruct corrupted inputs. It combines pretraining schemes of BERT and GPT-2.

In [ ]:
pipe = pipeline('summarization', model='facebook/bart-large-cnn')
pipe_out = pipe(sample_text)
summaries['bart'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

### PEAGSUS

PEAGSUS is also an encoder-decoder architecture that is based on the premise that the closer the pretraining objective is to the downstream task, the more effectifve it is. In a very large corpus, sentences containing most of the content in their surrounding paragraphs can be reconstructed to obtain a SOTA model for text summarization.

In [ ]:
pipe = pipeline('summarization', model='google/pegasus-pubmed')
pipe_out = pipe(sample_text)
summaries['pegasus'] = pipe_out[0]['summary_text'].replace(' .<n>', '.\n')

### Comparing Generated Summaries

In [ ]:
print(colored('Original', 'red'))
print(colored(dataset['train'][art_idx]['abstract'], 'green'))
print('')

for model_name in summaries:
  print(colored(model_name.upper(), 'red'))
  print(colored(summaries[model_name], 'blue'))
  print('')

### Evaluating using ROGUE Metric

The ROUGE score was developed for applications like summarization where high recall is more important than precision. ROUGE is calculated based on how many `n`-grams in the reference text also occur in the generated text.

In [ ]:
rouge_metric = load_metric('rouge', chace_dir=None)
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

test_sampled = dataset['test'].shuffle(seed=42).select(range(250))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
rogue_scores = pd.DataFrame.from_dict(rouge_dict, orient='index', columns=['baseline']).T

In [ ]:
model_ckpt = "google/pegasus-pubmed"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size=4)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
rogue_scores = rogue_scores.append(pd.DataFrame(rouge_dict, index=["pegasus"]))
rogue_scores

## HuggingFace's QA pipeline

### Load test dataset using HuggingFace data processors

The test dataset we are using for this task is the SQuAD-1.1 MRC dataset which consists of 10,570 question-answer pairs and 48 unique contexts.

[Stanford Question Answering Dataset (SQuAD)](https://arxiv.org/abs/1606.05250) is the first large-scale extractive MRC dataset. It was built by crowd-workers using a set of Wikipedia articles where the answer to each question is an extracted slice from the corresponding text passage.

This dataset can be downloaded from the Hugging Face hub [here](https://huggingface.co/datasets/squad).

In [ ]:
path_data = "/net/kdinxidk03/opt/NFS/collab_dir/SIGIR2022/"
processor = SquadV1Processor()
examples = processor.get_dev_examples(path_data, filename="squad/test_squad.json")

In [ ]:
print('An example sample in the SQuAD-1.1 development set: ',
      '\n\nQuestion: ', examples[10].__dict__['question_text'],
      '\n\nContext: ', examples[10].__dict__['context_text'],
      '\n\nAnswer: ', examples[10].__dict__['answers'][0]['text'],
     )

In [ ]:
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
answer_qids = list(qid_to_example_index.keys())

### Load a transformer model fine-tuned on SQuAD-1.1 for inference

In [ ]:
model_name_or_path = "csarron/bert-base-uncased-squad-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_name_or_path)

### Get prediction and evaluate the prediction using EM & F1 metrics

* Metrics for QA:

1. Exact Match (EM): For each question+answer pair, if the _characters_ of the model's prediction exactly match the characters of (one of) the True Answer(s), EM = 1, otherwise EM = 0. This is a strict all-or-nothing metric; being off by a single character results in a score of 0.

2. F1: F1 score is a common metric for classification problems, and widely used in QA. It is appropriate when we care equally about precision and recall. In this case, it's computed over the individual _words_ in the prediction against those in the True Answer. The number of shared words between the prediction and the truth is the basis of the F1 score: precision is the ratio of the number of shared words to the total number of words in the _prediction_, and recall is the ratio of the number of shared words to the total number of words in the _ground truth_.

In [ ]:
idx = 200
prediction = get_prediction(answer_qids[idx])
example = examples[qid_to_example_index[answer_qids[idx]]]
gold_answers = get_gold_answers(example)

em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

print(f"\nQuestion: {example.question_text}")
print(f"\nContext: {example.context_text}")
print(f"\nPrediction: {prediction}")
print(f"\nTrue Answers: {gold_answers}")
# print(f"\nPerformance Scores: EM: {em_score} \t F1: {f1_score}")